In [1]:
import pytools.leantools as lt
import pandas as pd
import os

In [2]:
script_dir = os.path.abspath('') #<-- absolute dir the script is in
result_dir = os.path.join (script_dir, "../results/parallax/backtest2")
algo_result_filepath = os.path.join(result_dir, "USDCAD-D1.json")
analysis_data_filepath = os.path.join(result_dir, "USDCAD-D1-analysis_data.json")
SYMBOLS = ['EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD', 'NZDUSD']
analysis_file_suffix = "-analysis_data"


In [3]:
stat_columns = ['TradeStats.TotalNumberOfTrades', 
             'TradeStats.ProfitLossRatio', 
             'PortfolioStats.AverageWinRate', 
             'PortfolioStats.AverageLossRate',
             'PortfolioStats.WinRate',
             'PortfolioStats.Drawdown',
            'PortfolioStats.TotalNetProfit']

def get_stats(symbols, timeframe):
    stats_list = []
    for s in symbols:
        name = f'{s}-{timeframe}'
        path = os.path.join(result_dir, f'{name}.json')
        stats = lt.get_trade_statistics(path)
        stats = stats[stat_columns]
        stats.rename(index={0:name}, inplace=True)
        stats_list.append(stats)

    result = pd.concat(stats_list)
    return result

https://stackoverflow.com/questions/38107304/pandas-changing-a-specific-row-to-percentages

In [4]:
# Get H4 stats
h4_stats = get_stats(SYMBOLS, "H4")
#h6_stats = get_stats(SYMBOLS, "H6")
d1_stats = get_stats(SYMBOLS, "D1")
all_stats = pd.concat([h4_stats, d1_stats])
#all_stats.style.format({
#        'PortfolioStats.WinRate' : '{:,.1%}'.format
#    })

def format_stats(stats_df):
    result = stats_df.transpose()
    result.loc['PortfolioStats.AverageWinRate'] = result.loc['PortfolioStats.AverageWinRate'].apply('{:,.1%}'.format)
    result.loc['PortfolioStats.AverageLossRate'] = result.loc['PortfolioStats.AverageLossRate'].apply('{:,.1%}'.format)
    result.loc['PortfolioStats.WinRate'] = result.loc['PortfolioStats.WinRate'].apply('{:,.1%}'.format)
    result.loc['PortfolioStats.Drawdown'] = result.loc['PortfolioStats.Drawdown'].apply('{:,.1%}'.format)
    result.loc['PortfolioStats.TotalNetProfit'] = result.loc['PortfolioStats.TotalNetProfit'].apply('{:,.1%}'.format)
    return result

#all_stats = format_stats(all_stats)
#all_stats

AttributeError: module 'pandas' has no attribute 'json_normalize'

In [189]:
#format_stats(h4_stats)

In [187]:
format_stats(d1_stats)

,EURUSD-D1,USDJPY-D1,GBPUSD-D1,USDCHF-D1,USDCAD-D1,AUDUSD-D1,NZDUSD-D1
TradeStats.TotalNumberOfTrades,18,21,18,17,20,27,24
TradeStats.ProfitLossRatio,2.4751,2.4028,2.887,3.2978,1.9677,2.8536,1.4099
PortfolioStats.AverageWinRate,1.9%,2.1%,2.1%,1.4%,1.7%,1.9%,1.2%
PortfolioStats.AverageLossRate,-0.8%,-0.9%,-0.7%,-0.4%,-0.8%,-0.7%,-0.8%
PortfolioStats.WinRate,38.9%,19.1%,33.3%,23.5%,40.0%,33.3%,33.3%
PortfolioStats.Drawdown,5.7%,6.3%,3.4%,4.6%,3.6%,6.8%,7.8%
PortfolioStats.TotalNetProfit,4.7%,-6.2%,3.9%,0.1%,3.2%,5.3%,-3.9%


In [4]:
bar_data_df = lt.get_bar_data_df(analysis_data_filepath)
#bar_data_df

In [35]:
closed_trades = lt.get_closed_trades_df(algo_result_filepath).drop(columns=['Direction', 'TotalFees', 'MAE', 'MFE', 'Symbol.ID', 'Symbol.Permtick'])
#closed_trades

Grab the trade signals

In [192]:
trade_setups_df = lt.get_trade_setups_df(analysis_data_filepath)
trade_setups_df

,symbol,direction,slPips,tpPips,plPips,tradeIndex
BarTime,,,,,,
2016-02-04 22:00:00+00:00,USDCAD,Buy,115.9,390.7,0.0,0
2016-03-07 22:00:00+00:00,USDCAD,Buy,79.7,268.9,-79.7,1
2016-03-20 21:00:00+00:00,USDCAD,Buy,54.3,182.9,182.9,2
2016-04-20 21:00:00+00:00,USDCAD,Buy,67.4,227.0,-68.1,3
2016-06-09 21:00:00+00:00,USDCAD,Buy,67.1,226.2,73.3,4
2016-11-14 22:00:00+00:00,USDCAD,Sell,65.3,220.1,-66.0,5
2016-12-13 22:00:00+00:00,USDCAD,Buy,118.1,397.8,0.0,-1
2017-06-19 21:00:00+00:00,USDCAD,Buy,44.6,150.3,0.0,-1
2018-01-08 22:00:00+00:00,USDCAD,Buy,44.4,149.6,66.6,6


In [193]:
merged_trade_data = trade_setups_df.join(closed_trades, on='tradeIndex')
merged_trade_data

,symbol,direction,slPips,tpPips,plPips,tradeIndex,EntryTime,EntryPrice,Quantity,ExitTime,ExitPrice,ProfitLoss,Duration,EndTradeDrawdown,Symbol.Value
BarTime,,,,,,,,,,,,,,,
2016-02-04 22:00:00+00:00,USDCAD,Buy,115.9,390.7,0.0,0,2016-01-12T14:06:00Z,1.41894,475973.0,2016-01-12T14:34:00Z,1.42199,-1020.98,00:28:00,-1148.18,USDCAD
2016-03-07 22:00:00+00:00,USDCAD,Buy,79.7,268.9,-79.7,1,2016-02-03T10:09:00Z,1.40293,314761.0,2016-02-03T13:29:00Z,1.38805,3374.52,03:20:00,-23.81,USDCAD
2016-03-20 21:00:00+00:00,USDCAD,Buy,54.3,182.9,182.9,2,2016-02-05T09:46:00Z,1.37137,304091.0,2016-02-05T13:34:00Z,1.37657,1148.40,03:48:00,-649.28,USDCAD
2016-04-20 21:00:00+00:00,USDCAD,Buy,67.4,227.0,-68.1,3,2016-02-11T18:01:00Z,1.39509,425718.0,2016-02-11T19:19:00Z,1.39850,-1038.12,01:18:00,-1615.03,USDCAD
2016-06-09 21:00:00+00:00,USDCAD,Buy,67.1,226.2,73.3,4,2016-02-18T10:03:00Z,1.36874,667900.0,2016-02-18T11:13:00Z,1.36662,-1036.01,01:10:00,-1532.02,USDCAD
2016-11-14 22:00:00+00:00,USDCAD,Sell,65.3,220.1,-66.0,5,2016-03-07T06:12:00Z,1.33390,873535.0,2016-03-07T11:24:00Z,1.33566,1150.98,05:12:00,-889.39,USDCAD
2016-12-13 22:00:00+00:00,USDCAD,Buy,118.1,397.8,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-06-19 21:00:00+00:00,USDCAD,Buy,44.6,150.3,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-08 22:00:00+00:00,USDCAD,Buy,44.4,149.6,66.6,6,2016-03-29T05:11:00Z,1.31964,1736531.0,2016-03-29T06:16:00Z,1.32051,1143.76,01:05:00,-493.01,USDCAD


In [8]:
#bar_data_df.join(trade_setups_df)

https://pandas.pydata.org/docs/user_guide/merging.html#timeseries-friendly-merging

In [202]:
merged_setups = pd.merge_asof(merged_trade_data, bar_data_df, left_on='BarTime', right_on='Time', direction='nearest')
merged_setups['range'] = merged_setups.apply(lambda row: (row.H - row.L) / 0.0001, axis = 1)
merged_setups['rangeAtrRatio'] = merged_setups.apply(lambda row: row.range / row.atrPips, axis = 1)
merged_setups

,symbol,direction,slPips,tpPips,plPips,tradeIndex,EntryTime,EntryPrice,Quantity,ExitTime,...,StochK,StochD,atrPips,O,H,L,C,spread,range,rangeAtrRatio
0,USDCAD,Buy,115.9,390.7,0.0,0,2016-01-12T14:06:00Z,1.41894,475973.0,2016-01-12T14:34:00Z,...,21.8028,15.9420,68.1,1.375425,1.376320,1.373255,1.374130,2.8,30.65,0.450073
1,USDCAD,Buy,79.7,268.9,-79.7,1,2016-02-03T10:09:00Z,1.40293,314761.0,2016-02-03T13:29:00Z,...,12.0919,16.6632,44.3,1.328290,1.331070,1.328025,1.331035,1.9,30.45,0.687359
2,USDCAD,Buy,54.3,182.9,182.9,2,2016-02-05T09:46:00Z,1.37137,304091.0,2016-02-05T13:34:00Z,...,21.7244,15.8540,63.5,1.300370,1.306665,1.300370,1.305565,2.5,62.95,0.991339
3,USDCAD,Buy,67.4,227.0,-68.1,3,2016-02-11T18:01:00Z,1.39509,425718.0,2016-02-11T19:19:00Z,...,19.4339,16.0261,60.8,1.265485,1.266085,1.264255,1.266060,2.0,18.30,0.300987
4,USDCAD,Buy,67.1,226.2,73.3,4,2016-02-18T10:03:00Z,1.36874,667900.0,2016-02-18T11:13:00Z,...,47.7677,44.9767,39.6,1.271995,1.274305,1.271805,1.273620,2.4,25.00,0.631313
5,USDCAD,Sell,65.3,220.1,-66.0,5,2016-03-07T06:12:00Z,1.33390,873535.0,2016-03-07T11:24:00Z,...,79.6567,83.8657,48.4,1.356130,1.356300,1.352915,1.353720,3.0,33.85,0.699380
6,USDCAD,Buy,118.1,397.8,0.0,-1,NaN,NaN,NaN,NaN,...,28.5123,19.2189,26.6,1.312945,1.313955,1.312875,1.313380,2.2,10.80,0.406015
7,USDCAD,Buy,44.6,150.3,0.0,-1,NaN,NaN,NaN,NaN,...,27.4329,25.7907,32.6,1.321975,1.322935,1.321485,1.322670,1.6,14.50,0.444785
8,USDCAD,Buy,44.4,149.6,66.6,6,2016-03-29T05:11:00Z,1.31964,1736531.0,2016-03-29T06:16:00Z,...,36.9133,36.0955,32.5,1.242205,1.242700,1.240995,1.241550,1.6,17.05,0.524615
9,USDCAD,Sell,61.9,208.7,-61.9,7,2016-03-30T05:02:00Z,1.30765,1160715.0,2016-03-30T06:01:00Z,...,64.8096,71.2614,30.9,1.270555,1.271845,1.269565,1.270745,1.9,22.80,0.737864


In [207]:
#merged_closed_trades = merged_setups
winning_trades = merged_setups[(merged_setups['plPips'] > 0) & (merged_setups['tradeIndex'] >= 0)]
losing_trades = merged_setups[(merged_setups['plPips'] <= 0) & (merged_setups['tradeIndex'] >= 0)]
describeColumns = ['slPips', 'tpPips', 'atrPips', 'spread', 'rangeAtrRatio']
winning_trades_desc = winning_trades.describe()[describeColumns]
losing_trades_desc = losing_trades.describe()[describeColumns]

#winning_trades_desc
winning_trades

,symbol,direction,slPips,tpPips,plPips,tradeIndex,EntryTime,EntryPrice,Quantity,ExitTime,...,StochK,StochD,atrPips,O,H,L,C,spread,range,rangeAtrRatio
2,USDCAD,Buy,54.3,182.9,182.9,2,2016-02-05T09:46:00Z,1.37137,304091.0,2016-02-05T13:34:00Z,...,21.7244,15.8540,63.5,1.300370,1.306665,1.300370,1.305565,2.5,62.95,0.991339
4,USDCAD,Buy,67.1,226.2,73.3,4,2016-02-18T10:03:00Z,1.36874,667900.0,2016-02-18T11:13:00Z,...,47.7677,44.9767,39.6,1.271995,1.274305,1.271805,1.273620,2.4,25.00,0.631313
8,USDCAD,Buy,44.4,149.6,66.6,6,2016-03-29T05:11:00Z,1.31964,1736531.0,2016-03-29T06:16:00Z,...,36.9133,36.0955,32.5,1.242205,1.242700,1.240995,1.241550,1.6,17.05,0.524615
12,USDCAD,Buy,42.3,142.4,47.5,10,2016-05-18T18:01:00Z,1.29287,298590.0,2016-05-18T18:05:00Z,...,21.0312,17.2113,33.9,1.282300,1.283035,1.281600,1.282780,1.8,14.35,0.423304
14,USDCAD,Sell,30.8,152.3,76.1,12,2016-07-06T09:08:00Z,1.29987,756201.0,2016-07-06T10:27:00Z,...,88.8492,85.6082,23.9,1.345605,1.345945,1.344430,1.345245,1.7,15.15,0.633891
16,USDCAD,Buy,39.5,133.2,61.4,14,2016-07-28T13:01:00Z,1.31599,592006.0,2016-07-29T01:21:00Z,...,51.6343,36.1124,23.0,1.328405,1.329195,1.327835,1.328020,1.6,13.60,0.591304
18,USDCAD,Sell,26.4,89.0,27.6,16,2016-08-29T21:22:00Z,1.30153,1022394.0,2016-08-30T01:21:00Z,...,44.1396,51.2963,16.7,1.316315,1.316790,1.315945,1.316410,1.6,8.45,0.505988
20,USDCAD,Buy,30.4,150.2,33.7,18,2016-10-05T20:00:00Z,1.31790,339759.0,2016-10-06T13:19:00Z,...,19.4447,21.1664,21.7,1.315105,1.315140,1.314155,1.314785,1.9,9.85,0.453917


In [208]:
losing_trades

,symbol,direction,slPips,tpPips,plPips,tradeIndex,EntryTime,EntryPrice,Quantity,ExitTime,...,StochK,StochD,atrPips,O,H,L,C,spread,range,rangeAtrRatio
0,USDCAD,Buy,115.9,390.7,0.0,0,2016-01-12T14:06:00Z,1.41894,475973.0,2016-01-12T14:34:00Z,...,21.8028,15.9420,68.1,1.375425,1.376320,1.373255,1.374130,2.8,30.65,0.450073
1,USDCAD,Buy,79.7,268.9,-79.7,1,2016-02-03T10:09:00Z,1.40293,314761.0,2016-02-03T13:29:00Z,...,12.0919,16.6632,44.3,1.328290,1.331070,1.328025,1.331035,1.9,30.45,0.687359
3,USDCAD,Buy,67.4,227.0,-68.1,3,2016-02-11T18:01:00Z,1.39509,425718.0,2016-02-11T19:19:00Z,...,19.4339,16.0261,60.8,1.265485,1.266085,1.264255,1.266060,2.0,18.30,0.300987
5,USDCAD,Sell,65.3,220.1,-66.0,5,2016-03-07T06:12:00Z,1.33390,873535.0,2016-03-07T11:24:00Z,...,79.6567,83.8657,48.4,1.356130,1.356300,1.352915,1.353720,3.0,33.85,0.699380
9,USDCAD,Sell,61.9,208.7,-61.9,7,2016-03-30T05:02:00Z,1.30765,1160715.0,2016-03-30T06:01:00Z,...,64.8096,71.2614,30.9,1.270555,1.271845,1.269565,1.270745,1.9,22.80,0.737864
10,USDCAD,Sell,112.7,380.0,-114.6,8,2016-04-06T06:35:00Z,1.31460,463458.0,2016-04-06T08:19:00Z,...,79.8463,81.2824,30.9,1.302155,1.303320,1.301145,1.303040,2.0,21.75,0.703883
11,USDCAD,Sell,69.8,235.4,-78.7,9,2016-04-27T18:07:00Z,1.26273,406304.0,2016-04-27T20:43:00Z,...,71.1251,79.7741,29.8,1.331840,1.332360,1.330035,1.330235,1.9,23.25,0.780201
13,USDCAD,Sell,51.6,174.0,-52.1,11,2016-06-24T14:02:00Z,1.29817,231418.0,2016-06-24T19:14:00Z,...,69.5657,67.7511,27.4,1.324295,1.324970,1.323490,1.324435,1.7,14.80,0.540146
15,USDCAD,Sell,25.4,85.7,0.0,13,2016-07-22T13:02:00Z,1.30851,415685.0,2016-07-22T13:56:00Z,...,70.3925,78.0633,24.0,1.348535,1.348805,1.347555,1.347895,1.7,12.50,0.520833
17,USDCAD,Buy,38.5,129.7,-43.0,15,2016-08-23T01:31:00Z,1.29301,919901.0,2016-08-23T06:31:00Z,...,36.7265,27.9465,23.8,1.309410,1.309660,1.307535,1.309455,1.7,21.25,0.892857


In [10]:
merged_setups.describe()

,slPips,tpPips,plPips,tradeIndex,EntryPrice,Quantity,ExitPrice,ProfitLoss,EndTradeDrawdown,BBMid,BBUpper,BBLower,StochK,StochD,atrPips,O,H,L,C,spread
count,117.000000,117.000000,117.000000,117.000000,95.000000,9.500000e+01,95.000000,95.000000,95.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000
mean,20.001709,75.623077,-4.800855,37.974359,1.311177,7.699300e+05,1.310932,-307.886947,-1131.866842,1.311597,1.319514,1.303685,44.933865,45.202174,33.976068,1.310802,1.312888,1.308911,1.310770,2.336752
std,11.156852,40.344556,26.932228,31.155467,0.035227,4.401622e+05,0.035449,1026.662346,519.395199,0.034538,0.034879,0.034650,34.588505,37.243885,11.305306,0.034186,0.033985,0.033916,0.033780,1.648332
min,2.900000,9.700000,-50.200000,-1.000000,1.235810,1.996440e+05,1.233830,-1593.780000,-2539.270000,1.232900,1.243200,1.222500,5.845300,4.350000,16.500000,1.233615,1.236640,1.233105,1.236440,1.400000
25%,11.300000,45.200000,-19.800000,7.000000,1.292940,4.599550e+05,1.292100,-930.485000,-1492.210000,1.291900,1.299100,1.282400,13.888900,11.045100,26.100000,1.294640,1.296340,1.292415,1.292965,1.600000
50%,17.800000,66.600000,-8.400000,36.000000,1.314780,6.679000e+05,1.313780,-783.850000,-1076.890000,1.315000,1.321800,1.307400,21.729200,17.933500,32.600000,1.314115,1.316395,1.313180,1.315090,1.800000
75%,26.900000,99.900000,0.000000,65.000000,1.332010,1.029104e+06,1.331690,-14.855000,-787.125000,1.331700,1.339600,1.326800,81.010000,84.690600,39.200000,1.330370,1.332930,1.329110,1.331795,2.300000
max,59.300000,199.700000,148.800000,94.000000,1.418940,3.249011e+06,1.421990,3594.090000,-23.810000,1.414500,1.426000,1.402900,95.022500,95.427600,71.700000,1.422370,1.423080,1.417665,1.418400,11.000000
